# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('temperatures.csv')
df = df.iloc[:, 1:]
df.head()

,City,Longitud,Latitude,Temperature,Humidity,Cloudiness,Wind_speed
0,Yellowknife,-114.3525,62.4560,-4.00,84,75,6.91
1,Coro,-69.6734,11.4045,72.14,87,8,12.44
2,Saint Paul Harbor,-152.4072,57.7900,32.00,43,1,24.16
3,Zyryanka,150.8500,65.7500,-26.97,94,100,4.09
4,Rikitea,-134.9692,-23.1203,78.49,72,85,14.79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = df[["Latitude", "Longitud"]]
humidity = df["Humidity"]



In [4]:
humidity.max()

100

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [56]:
#base_url = "https://maps.googleapis.com/maps/api/geocode/json?" 

#params = {"latlng":(-21.2078, -159.7750), "key": g_key}
#response = requests.get(base_url, params = params).json()

Narrowing down the DataFrame the ideal weather condition:

-A max temperature lower than 80 degrees but higher than 70.

-Wind speed less than 10 mph.

-Zero cloudiness.

In [6]:
ideal_df = df[(df['Temperature']<80) & (df['Temperature']>70) & (df['Wind_speed']<10) & (df['Cloudiness']==0)]

In [7]:
ideal_df

,City,Longitud,Latitude,Temperature,Humidity,Cloudiness,Wind_speed
22,Busselton,115.3333,-33.6500,73.35,56,0,5.99
55,Jaspur,78.8167,29.2833,78.80,44,0,5.75
84,Plettenberg Bay,23.3716,-34.0527,70.27,61,0,7.18
147,Port Elizabeth,25.5701,-33.9180,73.40,60,0,5.75
167,Cabo Rojo,-67.1457,18.0866,70.66,67,0,8.03
224,Santiago del Estero,-64.2615,-27.7951,73.00,69,0,3.00
228,Marzuq,46.4667,14.4000,79.77,18,0,8.81
294,São Filipe,-24.4956,14.8961,71.33,71,0,8.28
313,Ormara,64.6357,25.2088,78.26,38,0,7.40
360,Saldanha,17.9442,-33.0117,71.60,53,0,5.75


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df

In [9]:
hotel_df

,City,Longitud,Latitude,Temperature,Humidity,Cloudiness,Wind_speed
22,Busselton,115.3333,-33.6500,73.35,56,0,5.99
55,Jaspur,78.8167,29.2833,78.80,44,0,5.75
84,Plettenberg Bay,23.3716,-34.0527,70.27,61,0,7.18
147,Port Elizabeth,25.5701,-33.9180,73.40,60,0,5.75
167,Cabo Rojo,-67.1457,18.0866,70.66,67,0,8.03
224,Santiago del Estero,-64.2615,-27.7951,73.00,69,0,3.00
228,Marzuq,46.4667,14.4000,79.77,18,0,8.81
294,São Filipe,-24.4956,14.8961,71.33,71,0,8.28
313,Ormara,64.6357,25.2088,78.26,38,0,7.40
360,Saldanha,17.9442,-33.0117,71.60,53,0,5.75


In [10]:
lats = []
lngs = []
lat_lng = []

for row in hotel_df.iterrows():
    
    lats.append(row[1]['Latitude'])
    lngs.append(row[1]['Longitud'])

lat_lng = list(zip(lats, lngs))

In [11]:
len(lat_lng)

14

In [12]:
names = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# geocoordinates
#target_coordinates = []
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary

        
for target in lat_lng:
    params = {
    "location": f"{target[0]}, {target[1]}",
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key }
    try: 
        response = requests.get(base_url, params=params).json()
        names.append(response['results'][0]['name'])
    except: 
        names.append(None)
    

In [13]:
names

['The Sebel Busselton',
 'Sandhu Hotel & Restaurant',
 'The Plettenberg Hotel',
 'Island Way Villa',
 "Hotel Perichi's",
 'Hotel Altos del Estero',
 None,
 'Hotel Miramar Fogo Brava',
 'Alqadri hotel zero point ormara',
 'Hoedjiesbaai Hotel',
 None,
 None,
 'Yongjiang Hotel',
 'Hotel le Consulat/Camping Touristique']

In [14]:
hotel_df['Hotel Name'] = names
hotel_df

<ipython-input-14-0dbec5bcd668>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = names


,City,Longitud,Latitude,Temperature,Humidity,Cloudiness,Wind_speed,Hotel Name
22,Busselton,115.3333,-33.6500,73.35,56,0,5.99,The Sebel Busselton
55,Jaspur,78.8167,29.2833,78.80,44,0,5.75,Sandhu Hotel & Restaurant
84,Plettenberg Bay,23.3716,-34.0527,70.27,61,0,7.18,The Plettenberg Hotel
147,Port Elizabeth,25.5701,-33.9180,73.40,60,0,5.75,Island Way Villa
167,Cabo Rojo,-67.1457,18.0866,70.66,67,0,8.03,Hotel Perichi's
224,Santiago del Estero,-64.2615,-27.7951,73.00,69,0,3.00,Hotel Altos del Estero
228,Marzuq,46.4667,14.4000,79.77,18,0,8.81,None
294,São Filipe,-24.4956,14.8961,71.33,71,0,8.28,Hotel Miramar Fogo Brava
313,Ormara,64.6357,25.2088,78.26,38,0,7.40,Alqadri hotel zero point ormara
360,Saldanha,17.9442,-33.0117,71.60,53,0,5.75,Hoedjiesbaai Hotel


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_1 = hotel_df[["Latitude", "Longitud"]]

In [16]:
<dt>Country</dt><dd>{Country}</dd>

SyntaxError: invalid syntax (<ipython-input-16-daf6d11da24d>, line 1)

In [17]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
symbol_layer = gmaps.symbol_layer(
    locations_1, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=hotel_info)

fig.add_layer(symbol_layer)
fig.add_layer(heat_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

Here we can see the places with ideal weather to go for vacations
Some of them have no nearby hotel (less than 5km)